In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from config import *

CATEGORIES_REVERSE = {v: k for k, v in CATEGORIES_CODES.items()}

In [20]:
class DataLoader:
    __slots__ = ('images', '__index', 'labels', '_size', 'data')
    shape = (224, 224, 1)
    
    def __init__(self, _path: str) -> None:
        self.data = pd.read_csv(_path)
        self.__index, self._size = 0, self.data.shape[0]
        
        self.images = np.empty((self._size,) + self.shape, dtype='float16')
        self.labels = np.empty(self._size, dtype='uint8')

    def load_data(self, show_progress: bool) -> None:
        for _, _path, _categories in self.data.values:
            self.add_image(_path, _categories, show_progress)

    def add_image(self, _path: str, category: int, show_progress: bool) -> None:
        image_path = f"data/train/{CATEGORIES_REVERSE[category]}/{_path}"
        img = ~cv2.imread(image_path, 0) / 255
        
        self.images[self.__index] = np.asarray(img, dtype='float16').reshape(self.shape)
        self.labels[self.__index] = category
        self.__index += 1
        if self.__index == self._size:
            self.labels = to_categorical(self.labels)
            print(f"[+] DONE {self._size}/{self._size}")
            return
        
        if show_progress:
            print(f"[+] Progress: {self.__index}/{self._size}")
            
    def show_image(self, index: int) -> None:
        plt.imshow(self.images[index], cmap='gray')

In [21]:
train_set_path = 'data/processed/train_set.csv'
train_data = DataLoader(train_set_path)
train_data.load_data(show_progress=False)
#train_data.show_image(301)

[+] DONE 14629/14629


In [22]:
test_set_path = 'data/processed/test_set.csv'
test_data = DataLoader(test_set_path)
test_data.load_data(show_progress=False)
#test_data.show_image(300)

[+] DONE 770/770


In [51]:
class SketchClassificatorModel:
    __slots__ = ('_data', '_shape', 'num_classes', 'model', 'version')
    
    def __init__(self, data, _version: str) -> None:
        self._data = data
        self._shape = data.shape
        self.num_classes = len(CATEGORIES_CODES)
        self.model = models.Sequential()
        self.version = _version

    def build_model(self) -> None:
        activation, final_activation = 'relu', 'softmax'
        kernel_size, pool_size = (6, 6), (4, 4)
        
        self.model.add(layers.Conv2D(32, kernel_size, activation=activation, input_shape=self._shape))
        self.model.add(layers.MaxPooling2D(pool_size=pool_size))
        self.model.add(layers.Conv2D(64, kernel_size=kernel_size, activation=activation))
        self.model.add(layers.MaxPooling2D(pool_size=pool_size))
        self.model.add(layers.Conv2D(128, kernel_size=kernel_size, activation=activation))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(128, activation=activation))
        self.model.add(layers.Dense(self.num_classes, activation=final_activation))
        self.model.add(layers.Dropout(0.4))
        
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    def train(self) -> None:
        self.model.fit(self._data.images, self._data.labels, epochs=6, batch_size=64, validation_split=0.05)
    
    def save(self) -> None:
        self.model.save(f'models/model_v{self.version}.h5')

In [52]:
new_model_version = '0.3'   
sketch_model = SketchClassificatorModel(train_data, new_model_version)
sketch_model.build_model()
sketch_model.train()
sketch_model.save()

del sketch_model

Epoch 1/6
 59/218 [=======>......................] - ETA: 2:03 - loss: 7.7383 - accuracy: 0.2076

KeyboardInterrupt: 

In [37]:
for models in os.listdir('models'):
    model_test = load_model(f'models/{models}')
    print(f'Model version: {models[6:-3]}')
    test_loss, test_accuracy = model_test.evaluate(test_data.images, test_data.labels)

Model version: v0.1
25/25 [==============================] - 4s 167ms/step - loss: 2.4549 - accuracy: 0.6506
Model version: v0.2
25/25 [==============================] - 3s 125ms/step - loss: 1.2435 - accuracy: 0.7597
